In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import time
import pandas as pd
stime = time.time()
temp = pd.read_csv('/content/drive/MyDrive/BigData/1brc/measurements.txt', delimiter=";", names=['location', 'celsius'], header=None)
print(temp.groupby(['location']).agg(['min', 'mean', 'max']))
print(time.time()-stime)

              celsius                 
                  min       mean   max
location                              
Abha            -29.5  18.001155  71.6
Abidjan         -21.9  25.998862  76.3
Abéché          -16.8  29.399406  80.0
Accra           -26.1  26.394080  75.5
Addis Ababa     -34.2  15.992698  68.5
...               ...        ...   ...
Zagreb          -37.7  10.705807  61.9
Zanzibar City   -20.6  25.997325  78.5
Zürich          -42.5   9.299623  59.5
Ürümqi          -44.2   7.405673  54.6
İzmir           -30.5  17.892632  64.3

[413 rows x 3 columns]
295.7688024044037


In [4]:
import time
import duckdb
stime = time.time()
temp = duckdb.read_csv('/content/drive/MyDrive/BigData/1brc/measurements.txt', delimiter=";", names = ['location', 'temperature'])
duckdb.sql(
      ''' SELECT location, min(temperature), avg(temperature), max(temperature)
          from temp group by location''').show()
print(time.time()-stime)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────────┬──────────────────┬────────────────────┬──────────────────┐
│   location    │ min(temperature) │  avg(temperature)  │ max(temperature) │
│    varchar    │      double      │       double       │      double      │
├───────────────┼──────────────────┼────────────────────┼──────────────────┤
│ Dili          │            -22.1 │  26.60406087263433 │             76.6 │
│ Mek'ele       │            -29.2 │  22.69705816072825 │             71.6 │
│ Arkhangelsk   │            -53.0 │ 1.3057803327365867 │             49.5 │
│ Palm Springs  │            -27.3 │ 24.506017755634325 │             73.2 │
│ Mexicali      │            -31.9 │ 23.093575940023722 │             73.6 │
│ Roseau        │            -26.8 │  26.21431665258238 │             79.9 │
│ Iqaluit       │            -58.6 │ -9.295048421981095 │             41.1 │
│ Harbin        │            -45.8 │  5.005391114180374 │             55.6 │
│ Napier        │            -34.0 │  14.60711256863256 │             68.5 │

In [8]:
import time
import polars as pl
temp = pl.scan_csv('/content/drive/MyDrive/BigData/1brc/measurements.txt',  separator=";",
                    with_column_names= lambda cols:["location", "temperature"], has_header=False)

stime = time.time()
lazy_query = (
    temp.lazy().sql('''SELECT location, min(temperature) as min_temp,
                        avg(temperature) as avg_temp,
                        max(temperature) as max_temp from self
                        group by location ''')
)
result = lazy_query.collect(streaming=True)

print(result)
print(time.time()-stime)

shape: (413, 4)
┌────────────┬──────────┬───────────┬──────────┐
│ location   ┆ min_temp ┆ avg_temp  ┆ max_temp │
│ ---        ┆ ---      ┆ ---       ┆ ---      │
│ str        ┆ f64      ┆ f64       ┆ f64      │
╞════════════╪══════════╪═══════════╪══════════╡
│ New Delhi  ┆ -33.1    ┆ 25.001821 ┆ 73.6     │
│ Brisbane   ┆ -24.6    ┆ 21.404907 ┆ 71.3     │
│ Austin     ┆ -29.7    ┆ 20.701228 ┆ 68.2     │
│ Ifrane     ┆ -36.6    ┆ 11.394707 ┆ 64.6     │
│ Charlotte  ┆ -33.8    ┆ 16.089111 ┆ 66.9     │
│ …          ┆ …        ┆ …         ┆ …        │
│ Ségou      ┆ -22.6    ┆ 28.00203  ┆ 73.7     │
│ Willemstad ┆ -20.9    ┆ 28.005372 ┆ 75.9     │
│ Blantyre   ┆ -26.2    ┆ 22.196779 ┆ 76.5     │
│ Tbilisi    ┆ -36.3    ┆ 12.896636 ┆ 62.9     │
│ Auckland   ┆ -34.4    ┆ 15.192041 ┆ 68.7     │
└────────────┴──────────┴───────────┴──────────┘
97.8682267665863


In [6]:
import time
import polars as pl

stime = time.time()
print(pl.thread_pool_size())
temp = pl.read_parquet('/content/drive/MyDrive/BigData/1brc/measurements.parquet')
print(temp.group_by("location").agg(
    [pl.col("temperature").min().alias('min_temp'),
    pl.col("temperature").mean().alias('mean_temp'),
    pl.col("temperature").max().alias('max_temp')]))
print(time.time()-stime)

8
shape: (413, 4)
┌─────────────┬────────────┬───────────┬───────────┐
│ location    ┆ min_temp   ┆ mean_temp ┆ max_temp  │
│ ---         ┆ ---        ┆ ---       ┆ ---       │
│ str         ┆ f32        ┆ f64       ┆ f32       │
╞═════════════╪════════════╪═══════════╪═══════════╡
│ Da Lat      ┆ -32.5      ┆ 17.904293 ┆ 65.800003 │
│ Mexico City ┆ -32.599998 ┆ 17.497663 ┆ 66.900002 │
│ Yinchuan    ┆ -46.200001 ┆ 8.999447  ┆ 64.300003 │
│ Cotonou     ┆ -21.5      ┆ 27.199187 ┆ 79.800003 │
│ Yangon      ┆ -21.1      ┆ 27.501307 ┆ 75.900002 │
│ …           ┆ …          ┆ …         ┆ …         │
│ Damascus    ┆ -33.599998 ┆ 16.991982 ┆ 69.5      │
│ Sana'a      ┆ -33.400002 ┆ 19.991498 ┆ 69.699997 │
│ Willemstad  ┆ -20.9      ┆ 28.005262 ┆ 75.900002 │
│ Fresno      ┆ -36.0      ┆ 17.899188 ┆ 66.300003 │
│ Murmansk    ┆ -49.900002 ┆ 0.60043   ┆ 49.299999 │
└─────────────┴────────────┴───────────┴───────────┘
53.84968137741089
